In [1]:
import tensorflow as tf

## Create tf dataset from a list

In [2]:

daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset shapes: (), types: tf.int32>

## Iterate through tf dataset

In [5]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


## Iterate through elements as numpy elements

In [6]:

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


## Iterate through first n elements in tf dataset


In [7]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108



# Filter sales numbers that are < 0



In [8]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [10]:
tf_dataset=tf_dataset.map(lambda x:x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


In [11]:
tf_dataset=tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset

## **Batching**


In [12]:

for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1512 2232]
[2304 1584]
[2232 2448]


## **Perform all of the above operations in one shot**

In [13]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 2232]
[1584 2448]
[2232 2304]



## **Images**

loading ramdonm images from my drive


In [15]:
import zipfile

local_zip = '/content/drive/MyDrive/Colab Notebooks/Intel_Img_Class.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [16]:
images_ds = tf.data.Dataset.list_files('/content/seg_test/seg_test/*/*', shuffle=False)

In [17]:
image_count = len(images_ds)
image_count

3000

In [18]:
type(images_ds)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [19]:
for file in images_ds.take(5):
    print(file.numpy())

b'/content/seg_test/seg_test/buildings/20057.jpg'
b'/content/seg_test/seg_test/buildings/20060.jpg'
b'/content/seg_test/seg_test/buildings/20061.jpg'
b'/content/seg_test/seg_test/buildings/20064.jpg'
b'/content/seg_test/seg_test/buildings/20073.jpg'


In [20]:
class_name= ['building', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [22]:
#train test split 
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)
#shuffles data automatically 

In [23]:
len(train_ds),len(test_ds)

(2400, 600)

In [24]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [25]:
get_label("/content/seg_test/seg_test/buildings/20057.jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'buildings'>

In [26]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [27]:
img, label = process_image("/content/seg_test/seg_test/buildings/20057.jpg")
img.numpy()[:2]

array([[[ 87.822815, 114.72211 , 168.54114 ],
        [109.29944 , 134.8919  , 181.20837 ],
        [133.5661  , 157.97662 , 191.97375 ],
        [198.14099 , 218.3772  , 237.36456 ],
        [225.05127 , 240.71149 , 247.92242 ],
        [197.11987 , 208.17615 , 214.04962 ],
        [222.98334 , 231.13855 , 238.22656 ],
        [237.76825 , 241.11841 , 250.50824 ],
        [198.02197 , 203.32556 , 212.46729 ],
        [184.91895 , 193.64551 , 196.53583 ],
        [232.16357 , 242.13    , 243.14679 ],
        [222.78705 , 235.46497 , 244.27747 ],
        [138.28766 , 147.49652 , 177.0434  ],
        [128.4674  , 137.47559 , 169.27655 ],
        [179.37366 , 186.90948 , 209.33325 ],
        [198.78497 , 206.12091 , 223.12872 ],
        [211.77972 , 219.61566 , 231.43597 ],
        [226.00201 , 233.00781 , 240.96942 ],
        [225.883   , 232.78851 , 236.1875  ],
        [222.39484 , 225.80212 , 229.98755 ],
        [223.07977 , 225.50946 , 229.55634 ],
        [232.01099 , 233.01099 , 2

In [28]:
train_ds = train_ds.map(process_image) #returns multi list of image and label
test_ds = test_ds.map(process_image)

In [30]:
for image, label in train_ds.take(1):
    print("****\n",image)
    print("****\n",label)

****
 tf.Tensor(
[[[ 87.822815 114.72211  168.54114 ]
  [109.29944  134.8919   181.20837 ]
  [133.5661   157.97662  191.97375 ]
  ...
  [108.300415 149.39844  205.34943 ]
  [108.78381  149.08594  207.29034 ]
  [108.74957  148.90668  207.82812 ]]

 [[ 95.921936 122.170654 164.87646 ]
  [145.81854  169.36133  209.31891 ]
  [122.6142   144.29388  178.09637 ]
  ...
  [104.50922  147.05469  203.      ]
  [105.453125 146.74219  204.65253 ]
  [104.81903  147.39844  205.22656 ]]

 [[149.82391  173.79266  199.36652 ]
  [153.19159  174.448    203.15955 ]
  [137.45032  155.05682  186.86774 ]
  ...
  [101.060425 144.0362   201.71094 ]
  [101.73578  144.17688  202.02985 ]
  [101.09802  144.66193  202.14062 ]]

 ...

 [[197.28906  209.28906  224.33167 ]
  [197.14203  209.14203  223.14203 ]
  [196.28906  208.28906  222.28906 ]
  ...
  [222.65051  229.65051  239.65051 ]
  [224.30609  231.30609  241.30609 ]
  [226.01776  233.01776  243.01776 ]]

 [[194.51562  207.25781  223.8919  ]
  [194.44916  207.19

In [31]:
def scale(image, label):
    return image/255, label

In [32]:
train_ds = train_ds.map(scale)

In [33]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.3444032  0.4498906  0.66094565]
****Label:  b'buildings'
****Image:  [0.42521447 0.71541053 0.8676777 ]
****Label:  b'buildings'
****Image:  [0.88786596 0.9113954  0.9662973 ]
****Label:  b'buildings'
****Image:  [0.5637238  0.68137085 0.92450815]
****Label:  b'buildings'
****Image:  [0.1565547 0.317339  0.5683194]
****Label:  b'buildings'


# Again we can chain it like we did earlier.